In [ ]:
#Данных клиентов объединяет то, что они совершали снятие через банкоматы партнеров, айди банка в основном 3 и 5

In [80]:
!pip install mathplotlib
!pip install pandas
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('try1.csv',sep=';')
df.head()

,TIME,TRANSACTION_ID,AMOUNT,CLIENT_ID,BANK_ID,CITY,OPERATION_TYPE,FRAUD_FLAG
0,2023-05-01,525471381920,1560,BD308B1E6247DAD7E7FAC425AB3CADA464C26F2A0B47EB...,6,VSEVOLOZHSK,Снятие через банкоматы партнеров,0
1,2023-05-01,525471383640,1560,BD308B1E6247DAD7E7FAC425AB3CADA464C26F2A0B47EB...,6,VSEVOLOZHSK,Снятие через банкоматы партнеров,0
2,2023-05-01,525487382296,290,2E021B8D64532F869A14167A6075996B903D52ED66B3AD...,6,OBNINSK,Снятие через банкоматы партнеров,0
3,2023-05-01,525364464247,310,B27401D40DC566402D47BA342DDD6A1457E91F3D211D0B...,3,CHITA G,Пополнение через банкоматы партнеров,0
4,2023-05-01,525513617413,140,B566176D8952A9B1AA4A25D1574AE1CA9EA57C2AB4F29C...,2,DALNEGORSK,Снятие через чужие банкоматы,0


In [83]:
df['TIME'] = pd.to_datetime(df['TIME'])
fraud_clients = df[df['FRAUD_FLAG'] == 1]

In [122]:
fraud_characteristics = fraud_clients.groupby(['TIME' ,'TRANSACTION_ID','AMOUNT','CLIENT_ID', 'BANK_ID','CITY', 'OPERATION_TYPE']).size().reset_index(name='counts')
print("Характеристики фродовых клиентов:")
print(fraud_characteristics)


Характеристики фродовых клиентов:
          TIME  TRANSACTION_ID  AMOUNT  \
0   2023-07-01    540482085666    1610   
1   2023-07-01    540517177893      80   
2   2023-07-02    540636758790     470   
3   2023-07-02    540640594084     470   
4   2023-07-02    540645052881     500   
..         ...             ...     ...   
515 2023-07-31    547376385515     940   
516 2023-07-31    547376834131     470   
517 2023-07-31    547379048625     470   
518 2023-07-31    547379744940     470   
519 2023-07-31    547379745207     470   

                                             CLIENT_ID  BANK_ID  \
0    CE04E622F86F602C957FA020B1459F73ABBFF5A6A32746...        3   
1    CE04E622F86F602C957FA020B1459F73ABBFF5A6A32746...        3   
2    B9983326ADA8FB296B488236557DE7A1FADB49300C4C82...        3   
3    B9983326ADA8FB296B488236557DE7A1FADB49300C4C82...        3   
4    B9983326ADA8FB296B488236557DE7A1FADB49300C4C82...        3   
..                                                 ...     

In [124]:
similar_clients = df[(df['AMOUNT'].isin(fraud_clients['AMOUNT'])) &
                       (df['CITY'].isin(fraud_clients['CITY'])) &
                       (df['OPERATION_TYPE'].isin(fraud_clients['OPERATION_TYPE'])) &
                       (df['BANK_ID'].isin(fraud_clients['BANK_ID'])) &
                       (df['FRAUD_FLAG'] == 0)]

In [125]:
df['MONTH'] = pd.to_datetime(df['TIME']).dt.to_period('M')
similar_clients['MONTH'] = pd.to_datetime(similar_clients['TIME']).dt.to_period('M')


C:\Users\Валерия\AppData\Local\Temp\ipykernel_13916\2813003114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_clients['MONTH'] = pd.to_datetime(similar_clients['TIME']).dt.to_period('M')


In [118]:
monthly_analysis = similar_clients.groupby('MONTH').size() / df.groupby('MONTH').size()
monthly_analysis = monthly_analysis.reset_index(name='FRAUD_CLIENT_SHARE')

print("Доля операций аналогичных клиентов по месяцам:")
print(monthly_analysis)


Доля операций аналогичных клиентов по месяцам:
     MONTH  FRAUD_CLIENT_SHARE
0  2023-05            0.082367
1  2023-06            0.082421
2  2023-07            0.079852
